In [26]:
#Module Imports
import pickle
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, normalize
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import sys, os
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from ploter import PlotDataFrame
from utils import Util
warnings.filterwarnings('ignore')

In [2]:
# Define the file path
file_path = '../data/cleaned_data.csv'

# Load the cleaned data into a DataFrame
cleaned_df = pd.read_csv(file_path)
cleaned_df.describe()

,Unnamed: 0,Bearer Id,Start ms,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Avg RTT DL (ms),Avg RTT UL (ms),...,Total UL (Bytes),Total DL (Bytes),Social Media Data Volume (Bytes),Google Data Volume (Bytes),Email Data Volume (Bytes),Youtube Data Volume (Bytes),Netflix Data Volume (Bytes),Gaming Data Volume (Bytes),Other Data Volume (Bytes),Total Data Volume (Bytes)
count,146887.000000,1.468870e+05,146887.000000,146887.000000,146887.000000,1.468870e+05,1.468870e+05,1.468870e+05,146887.000000,146887.000000,...,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05,1.468870e+05
mean,75229.065050,-8.633563e+17,499.117757,498.657505,100188.834299,2.082017e+14,3.596861e+10,4.844173e+13,54.044476,9.300626,...,4.112139e+07,4.546601e+08,1.827736e+06,7.807976e+06,2.259087e+06,2.264198e+07,2.262923e+07,4.303526e+08,4.292669e+08,4.957815e+08
std,43252.410975,8.256071e+18,288.557730,288.073845,58988.245241,1.334727e+09,8.792146e+11,2.238836e+13,31.125458,9.610258,...,1.127057e+07,2.442101e+08,1.035571e+06,3.517132e+06,1.071133e+06,9.244659e+06,9.259113e+06,2.440925e+08,2.432060e+08,2.444434e+08
min,0.000000,-9.223372e+18,0.000000,0.000000,7142.000000,2.082001e+14,3.360100e+10,4.400152e+11,0.000000,0.000000,...,9.493545e+06,7.114041e+06,1.563000e+03,4.033000e+04,8.359000e+03,1.296550e+05,9.843200e+04,3.063580e+05,1.490450e+05,2.895611e+07
25%,37949.500000,-9.223372e+18,250.000000,251.000000,58526.500000,2.082014e+14,3.365128e+10,3.546071e+13,31.000000,2.000000,...,3.321835e+07,2.430734e+08,9.313390e+05,4.943585e+06,1.359012e+06,1.599692e+07,1.598200e+07,2.186904e+08,2.184772e+08,2.844777e+08
50%,75110.000000,6.917538e+18,499.000000,500.000000,86399.000000,2.082015e+14,3.366369e+10,3.572181e+13,44.000000,5.000000,...,4.114314e+07,4.559639e+08,1.825390e+06,7.815531e+06,2.263890e+06,2.266341e+07,2.263459e+07,4.316603e+08,4.297978e+08,4.969362e+08
75%,112724.500000,7.349883e+18,749.000000,750.000000,133159.000000,2.082018e+14,3.368344e+10,8.611970e+13,67.000000,13.000000,...,4.903488e+07,6.657831e+08,2.727156e+06,1.068314e+07,3.159616e+06,2.929351e+07,2.928806e+07,6.415752e+08,6.396625e+08,7.066379e+08
max,149995.000000,7.566056e+18,999.000000,999.000000,245107.750000,2.082099e+14,3.370000e+14,9.900120e+13,121.000000,29.500000,...,7.275968e+07,9.029696e+08,3.650861e+06,1.552878e+07,4.518036e+06,4.519008e+07,4.519815e+07,8.592028e+08,8.595209e+08,9.536316e+08


In [3]:

# Select Necessary Columns
df_user_experience = cleaned_df[[
    'MSISDN/Number',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)',
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Handset Type']]
df_user_experience


,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type
0,33664962239,42.0,5.0,23.0,44.0,19520.0,7230.0,Samsung Galaxy A5 Sm-A520F
1,33681854413,65.0,5.0,16.0,26.0,19520.0,7230.0,Samsung Galaxy J5 (Sm-J530)
2,33760627129,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Galaxy A8 (2018)
3,33750343200,65.0,5.0,44.0,44.0,19520.0,7230.0,undefined
4,33699795932,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Sm-G390F
...,...,...,...,...,...,...,...,...
146882,33762146363,27.0,2.0,62.0,54.0,7658052.5,162614.0,Apple iPhone 6S Plus (A1687)
146883,33761274518,37.0,4.0,23.0,40.0,7658052.5,162614.0,Apple iPhone 6S (A1688)
146884,33626114091,46.0,6.0,43.0,41.0,7658052.5,162614.0,Apple iPhone Xr (A2105)
146885,33645655643,46.0,6.0,55.0,54.0,7658052.5,162614.0,Samsung Galaxy J5 (Sm-J530)


In [4]:
#Caluclating the totals
df_user_experience['Total Avg RTT (ms)'] = df_user_experience['Avg RTT DL (ms)'] + df_user_experience['Avg RTT UL (ms)']
df_user_experience['Total Avg Bearer TP (kbps)'] = df_user_experience['Avg Bearer TP DL (kbps)'] + df_user_experience['Avg Bearer TP UL (kbps)']
df_user_experience['Total TCP Retrans. Vol (Bytes)'] = df_user_experience['TCP DL Retrans. Vol (Bytes)'] + df_user_experience['TCP UL Retrans. Vol (Bytes)']

df_user_experience

/tmp/ipykernel_10794/3626448605.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_experience['Total Avg RTT (ms)'] = df_user_experience['Avg RTT DL (ms)'] + df_user_experience['Avg RTT UL (ms)']
/tmp/ipykernel_10794/3626448605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_experience['Total Avg Bearer TP (kbps)'] = df_user_experience['Avg Bearer TP DL (kbps)'] + df_user_experience['Avg Bearer TP UL (kbps)']
/tmp/ipykernel_10794/3626448605.py:4: SettingWithCopyWarning: 
A value is tryi

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.0,5.0,23.0,44.0,19520.0,7230.0,Samsung Galaxy A5 Sm-A520F,47.0,67.0,26750.0
1,33681854413,65.0,5.0,16.0,26.0,19520.0,7230.0,Samsung Galaxy J5 (Sm-J530),70.0,42.0,26750.0
2,33760627129,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Galaxy A8 (2018),70.0,15.0,26750.0
3,33750343200,65.0,5.0,44.0,44.0,19520.0,7230.0,undefined,70.0,88.0,26750.0
4,33699795932,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Sm-G390F,70.0,15.0,26750.0
...,...,...,...,...,...,...,...,...,...,...,...
146882,33762146363,27.0,2.0,62.0,54.0,7658052.5,162614.0,Apple iPhone 6S Plus (A1687),29.0,116.0,7820666.5
146883,33761274518,37.0,4.0,23.0,40.0,7658052.5,162614.0,Apple iPhone 6S (A1688),41.0,63.0,7820666.5
146884,33626114091,46.0,6.0,43.0,41.0,7658052.5,162614.0,Apple iPhone Xr (A2105),52.0,84.0,7820666.5
146885,33645655643,46.0,6.0,55.0,54.0,7658052.5,162614.0,Samsung Galaxy J5 (Sm-J530),52.0,109.0,7820666.5


In [30]:
df_user_experience_metrics=df_user_experience[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
df_user_experience_metrics

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.0,67.0,26750.0,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.0,42.0,26750.0,Samsung Galaxy J5 (Sm-J530)
2,33760627129,70.0,15.0,26750.0,Samsung Galaxy A8 (2018)
3,33750343200,70.0,88.0,26750.0,undefined
4,33699795932,70.0,15.0,26750.0,Samsung Sm-G390F
...,...,...,...,...,...
146882,33762146363,29.0,116.0,7820666.5,Apple iPhone 6S Plus (A1687)
146883,33761274518,41.0,63.0,7820666.5,Apple iPhone 6S (A1688)
146884,33626114091,52.0,84.0,7820666.5,Apple iPhone Xr (A2105)
146885,33645655643,52.0,109.0,7820666.5,Samsung Galaxy J5 (Sm-J530)


## TCP Retransmission Volume

In [31]:
# Top 10, bottom 10, and most frequent TCP Retransmission Volume
print("TCP Retransmission Volume (Bytes):")
print("Top 10:")
df_user_experience_metrics.nlargest(10, 'Total TCP Retrans. Vol (Bytes)')



TCP Retransmission Volume (Bytes):
Top 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
14,33753758738,68.5,883.0,7824924.5,Samsung Galaxy J3 (Sm-J330)
21,33621235801,23.0,101.0,7824924.5,Apple iPhone 6S (A1688)
22,33681827486,23.0,16.0,7824924.5,Samsung Galaxy A3 (Sm-A320)
23,33611536843,23.0,122.0,7824924.5,Samsung Galaxy J3 (Sm-J330)
24,33770418553,67.0,57.0,7824924.5,Huawei P20 Pro
25,33662839194,39.0,79.0,7824924.5,Samsung Galaxy J3 (Sm-J330)
26,33664343412,39.0,15.0,7824924.5,Samsung Galaxy A3 (Sm-A320)
27,33669162989,36.0,33.0,7824924.5,Samsung Galaxy J5 (Sm-J530)
28,33698257107,86.5,17782.0,7824924.5,Samsung Sm-G390F
29,33658752999,143.0,28806.5,7824924.5,Huawei E5180


In [32]:
print("Bottom 10:")
df_user_experience_metrics.nsmallest(10, 'Total TCP Retrans. Vol (Bytes)')



Bottom 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
91377,33662545528,66.0,16850.5,51.0,undefined
2670,33761418628,42.0,8783.0,81.0,Apple iPhone 6 (A1586)
78606,33666767218,35.0,139.0,82.0,Apple iPhone 6 (A1586)
140565,33658794235,150.5,411.0,86.0,Huawei E5180
8591,33699259122,52.0,121.0,88.0,Apple iPhone 6S (A1688)
11575,33760003506,41.0,74.0,88.0,Samsung Galaxy Note9
11576,33661788769,41.0,318.0,88.0,Samsung Galaxy S7 (Sm-G930X)
58103,33669388268,44.0,157.0,92.0,Zte Mf259
108549,33658223831,29.0,15.0,97.0,Samsung Galaxy J3 (Sm-J330)
108550,33660364301,29.0,38.0,97.0,undefined


In [33]:
print("Most Frequent:")
df_user_experience_metrics['Total TCP Retrans. Vol (Bytes)'].value_counts().head(10)


Most Frequent:


Total TCP Retrans. Vol (Bytes)
7824924.5    8492
7659346.5     384
7659382.5     177
115067.0      150
1432.0        146
1441.0        142
2660.0        134
985475.0      119
520658.0      113
1328130.0     112
Name: count, dtype: int64

## Average RTT 

In [34]:

print("\nAverage RTT (ms):")
print("Top 10:")
df_user_experience_metrics.nlargest(10, 'Total Avg RTT (ms)')



Average RTT (ms):
Top 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
19,33658752999,150.5,32270.5,7679638.5,Huawei E5180
145,33663395710,150.5,48081.0,7706629.5,Huawei B528S-23A
149,33650392011,150.5,11481.0,2677382.0,Huawei E5180
163,33650059450,150.5,20648.5,29304.0,Huawei E5180
181,33627850141,150.5,89.0,167189.0,Apple iPhone 7 (A1778)
272,33658491763,150.5,3142.0,3067446.0,Huawei E5180
463,33660917355,150.5,20991.0,773611.0,Samsung Galaxy J5 (Sm-J530)
486,33672450536,150.5,543.0,1751539.0,Tcl Communicatio. Alcatel Pixi 4 (4) 4034D
505,33668290878,150.5,718.0,6361275.0,Samsung Galaxy Note 2 (Gt-N7100)
513,33617340134,150.5,12890.5,7686867.5,Wiko Global Sasu Wiko Lenny4


In [35]:
print("Bottom 10:")
df_user_experience_metrics.nsmallest(10, 'Total Avg RTT (ms)')


Bottom 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
41313,33652982188,0.0,100.0,63004.0,Huawei P20 Pro
100855,33609260285,0.0,99.0,336337.0,Samsung Galaxy S6 (Sm-G920X)
100856,33670954549,0.0,193.0,336337.0,Samsung Galaxy A8 (2018)
100857,33662760169,0.0,127.0,336337.0,Samsung Galaxy A8 (2018)
100858,33664514111,0.0,151.0,336337.0,Samsung Galaxy S9 Sm-G960F Ds
121762,33661575382,0.0,85.0,66178.0,Huawei Pct-L29
121763,33614382389,0.0,95.0,66178.0,Samsung Galaxy S9
140847,33642753301,0.0,100.0,6892607.0,Apple iPhone 6 (A1586)
140848,33637171012,0.0,118.0,6892607.0,undefined
70123,33660204450,2.0,113.0,24107.0,Apple iPhone X (A1901)


In [36]:
print("Most Frequent:")
df_user_experience_metrics['Total Avg RTT (ms)'].value_counts().head(10)

Most Frequent:


Total Avg RTT (ms)
29.0     6722
39.0     5411
38.0     3469
30.0     3448
150.5    3354
40.0     3337
28.0     3323
49.0     2728
31.0     2562
41.0     2442
Name: count, dtype: int64

## Average Bearer Throughput

In [37]:

print("\nAverage Bearer Throughput (kbps):")
print("Top 10:")
df_user_experience_metrics.nlargest(10, 'Total Avg Bearer TP (kbps)')



Average Bearer Throughput (kbps):
Top 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
77,33669165983,115.0,52765.0,658099.0,Samsung Galaxy S9 Plus (Sm-G965F)
85,33663676737,62.0,52765.0,25576.0,Apple iPad Pro 10.5 (A1709)
130,33669165983,90.0,52765.0,639492.0,Samsung Galaxy S9 Plus (Sm-G965F)
155,33663537175,84.0,52765.0,2962123.0,Huawei B528S-23A
167,33665771979,107.5,52765.0,7683563.5,Huawei B528S-23A
190,33660594196,58.5,52765.0,175043.0,undefined
312,33783350142,50.0,52765.0,173190.0,Huawei P8 Lite
318,33669165983,55.0,52765.0,26822.0,Samsung Galaxy S9 Plus (Sm-G965F)
326,33659675942,53.0,52765.0,7824924.5,Tcl Communicatio. Linkzone
432,33659074100,138.0,52765.0,7663222.5,Huawei P10


In [38]:
print("Bottom 10:")
df_user_experience_metrics.nsmallest(10, 'Total Avg Bearer TP (kbps)')


Bottom 10:


,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
604,33658211925,30.0,0.0,1858803.0,Wct Uno
2430,33761309821,49.5,0.0,7738023.5,Lg Lg-T385
9120,33650241905,125.0,0.0,6544490.0,Huawei P20 Lite Huawei Nova 3E
13597,33659471143,123.0,0.0,1970509.0,Huawei P20 Pro
14407,33634247429,74.0,0.0,1005249.0,Huawei Y6 2018
19740,33667336746,150.5,0.0,40887.0,undefined
20378,33604131582,59.0,0.0,167395.0,Lenovo Moto G (3Rd Gen) Ca12459445
23551,33650000334,28.0,0.0,3770.0,undefined
65363,33661089447,109.5,0.0,4673289.0,Huawei B528S-23A
67641,33616110252,40.0,0.0,152847.0,Apple iPhone Xr (A2105)


In [39]:
print("Most Frequent:")
df_user_experience_metrics['Total Avg Bearer TP (kbps)'].value_counts().head(10)

Most Frequent:


Total Avg Bearer TP (kbps)
52765.0    7622
63.0       3884
15.0       3678
97.0       1945
90.0       1881
98.0       1799
96.0       1669
99.0       1569
89.0       1554
91.0       1512
Name: count, dtype: int64

In [40]:
# Compute the distribution of average throughput per handset type
print("Distribution of Average Throughput per Handset Type:")
throughput_by_handset = df_user_experience_metrics.groupby('Handset Type')['Total Avg Bearer TP (kbps)'].describe()
throughput_by_handset


Distribution of Average Throughput per Handset Type:


,count,mean,std,min,25%,50%,75%,max
Handset Type,,,,,,,,
A-Link Telecom I. Cubot A5,1.0,23510.000000,NaN,23510.0,23510.0,23510.0,23510.0,23510.0
A-Link Telecom I. Cubot Note Plus,1.0,6699.000000,NaN,6699.0,6699.0,6699.0,6699.0,6699.0
A-Link Telecom I. Cubot Note S,1.0,8937.000000,NaN,8937.0,8937.0,8937.0,8937.0,8937.0
A-Link Telecom I. Cubot Nova,1.0,50545.500000,NaN,50545.5,50545.5,50545.5,50545.5,50545.5
A-Link Telecom I. Cubot Power,1.0,50337.500000,NaN,50337.5,50337.5,50337.5,50337.5,50337.5
...,...,...,...,...,...,...,...,...
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809,2.0,58.000000,73.539105,6.0,32.0,58.0,84.0,110.0
Zyxel Communicat. Lte7460,1.0,42510.500000,NaN,42510.5,42510.5,42510.5,42510.5,42510.5
Zyxel Communicat. Sbg3600,1.0,52765.000000,NaN,52765.0,52765.0,52765.0,52765.0,52765.0


In [41]:
# Compute the average TCP retransmission per handset type
print("\nAverage TCP Retransmission per Handset Type:")
tcp_retrans_by_handset = df_user_experience_metrics.groupby('Handset Type')['Total TCP Retrans. Vol (Bytes)'].mean()
tcp_retrans_by_handset



Average TCP Retransmission per Handset Type:


Handset Type
A-Link Telecom I. Cubot A5                                                             7.673106e+06
A-Link Telecom I. Cubot Note Plus                                                      6.162130e+05
A-Link Telecom I. Cubot Note S                                                         7.725304e+06
A-Link Telecom I. Cubot Nova                                                           1.371700e+05
A-Link Telecom I. Cubot Power                                                          8.041000e+03
                                                                                           ...     
Zte Zte Blade C2 Smartphone Android By Sfr Startrail 4 Zte Blade Flex T809 Zte T809    6.554150e+04
Zyxel Communicat. Lte7460                                                              7.824924e+06
Zyxel Communicat. Sbg3600                                                              7.824924e+06
Zyxel Communicat. Zyxel Wah7706                                                        

In [42]:
# Assuming the DataFrame is named df_user_experience_metrics
X = df_user_experience_metrics[['Total Avg RTT (ms)', 'Total Avg Bearer TP (kbps)', 'Total TCP Retrans. Vol (Bytes)']]

# Perform k-means clustering with k=3
kmeans = KMeans(n_clusters=3, random_state=42)
labels = kmeans.fit_predict(X)

# Add the cluster labels to the original DataFrame
df_user_experience_metrics['Cluster'] = labels




In [43]:
df_user_experience_metrics['Cluster'].value_counts()

Cluster
0    98782
1    27521
2    20584
Name: count, dtype: int64

In [44]:
# Describe the clusters
print("Cluster Analysis:")
for cluster in range(3):
    cluster_data = df_user_experience_metrics[df_user_experience_metrics['Cluster'] == cluster]
    
    print(f"Cluster {cluster}:")
    print(f"Average RTT: {cluster_data['Total Avg RTT (ms)'].mean():.2f} ms")
    print(f"Average Throughput: {cluster_data['Total Avg Bearer TP (kbps)'].mean():.2f} kbps")
    print(f"Average TCP Retransmission: {cluster_data['Total TCP Retrans. Vol (Bytes)'].mean():.2f} bytes")
    print(f"Handset Types: {', '.join(cluster_data['Handset Type'].unique())}")
    print()

Cluster Analysis:
Cluster 0:
Average RTT: 58.40 ms
Average Throughput: 9620.05 kbps
Average TCP Retransmission: 318803.92 bytes
Handset Types: Samsung Galaxy A5 Sm-A520F, Samsung Galaxy J5 (Sm-J530), Samsung Galaxy A8 (2018), undefined, Samsung Sm-G390F, Huawei Mate 20 Lite, Samsung Galaxy A3 (Sm-A320), Samsung Galaxy J7, Samsung Galaxy J3 (Sm-J330), Samsung Galaxy S7 (Sm-G930X), Zte Mf259, Tp-Link Technolo. Tp-Link Archer Mr200 Tp-Link Tl-Mr6400, Samsung Galaxy S9 Sm-G960F Ds, Huawei B528S-23A, Huawei Mate 10 Pro Porsche Design Huawei Mate 10, Apple iPhone 6S (A1688), Samsung Galaxy Note 8 (Sm-N950F), Apple iPhone Se (A1723), Apple iPhone 8 (A1905), Samsung Galaxy S10, Apple iPhone 7 (A1778), Samsung Galaxy S6 Edge (Sm-G925X), Samsung Galaxy A6 Sm-A600Fn Ds, Apple iPhone 7 Plus (A1784), Apple iPhone 6 (A1586), Apple iPhone 6 Plus (A1524), Apple iPhone Xr (A2105), Samsung Galaxy S9 Plus (Sm-G965F), Samsung Galaxy S8 (Sm-G950F), Apple iPad Pro 10.5 (A1709), Apple iPhone 6S (A1633), Huaw

In [45]:

# Specify the file path and name for the CSV file
df_user_experience_metrics.to_csv('../data/user_experience.csv')

#Saveing the clustering model
with open("../models/user_experience.pkl", "wb") as f:
    pickle.dump(kmeans, f)
